In [ ]:
import batoid
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import obscuration

In [ ]:
HSC_fn = os.path.join(batoid.datadir, "hsc", "HSC.yaml")
HSC = batoid.Telescope.makeFromYAML(HSC_fn)
HSC2 = batoid.Telescope.makeFromYAML(HSC_fn)

In [ ]:
for i, k in enumerate(HSC.surfaces):
    print(i, k)

In [ ]:
# So surface 15 is the filter entrance.  Let's use that.
# Use HSC2 so don't mess up HSC
HSC_front = batoid.Telescope(HSC2.surfaces[15:None:-1], name='HSC_front')
# Need to flip ingoing/outgoing media
for k, v in HSC_front.surfaces.items():
    v['m0'], v['m1'] = v['m1'], v['m0']
HSC_back = batoid.Telescope(HSC2.surfaces[16:], name='HSC_back')

In [ ]:
cameraShadow = obscuration.Circle(0.0, 0.0, 0.231*8.2*0.5)
alpha = (51.75-45)*np.pi/180
strut1 = obscuration.Ray(0.43, 0.43, 0.22, np.pi/2+alpha)
strut2 = obscuration.Ray(0.43, 0.43, 0.22, -alpha)
strut3 = obscuration.Ray(-0.43, -0.43, 0.22, np.pi-alpha)
strut4 = obscuration.Ray(-0.43, -0.43, 0.22, 3*np.pi/2+alpha)
spider = obscuration.Union(cameraShadow, strut1, strut2, strut3, strut4)

In [ ]:
# Function to trace rays through HSC_front:
def trace_front(x, y, vx, vy):
    wavelength = 750e-9
    z = HSC_front.surfaces[0]['surface'].B+0.001
    m = HSC_front.surfaces[0]['m0']
    vz = -1
    xvec = batoid.Vec3(x, y, z)
    vvec = batoid.Vec3(vx, vy, vz).UnitVec3()/m.getN(wavelength)
    r = batoid.Ray(xvec, vvec, t=0, w=wavelength)
    return HSC_front.trace(r)
def trace_back(x, y, vx, vy):
    wavelength = 750e-9
    z = HSC_front.surfaces[0]['surface'].B+0.001
    m = HSC_front.surfaces[0]['m0']
    vz = 1
    xvec = batoid.Vec3(x, y, z)
    vvec = batoid.Vec3(vx, vy, vz).UnitVec3()/m.getN(wavelength)
    r = batoid.Ray(xvec, vvec, t=0, w=wavelength)
    return HSC_back.trace(r)

In [ ]:
def pinhole(x, y):
    vxs = np.linspace(-0.2, 0.2, 400)
    vys = vxs
    fpx = []
    fpy = []
    pmx = []
    pmy = []
    for i in range(len(vxs)):
        for j in range(len(vys)):
            tf = trace_front(x, y, vxs[i], vys[j])
            tb = trace_back(x, y, vxs[i], vys[j])            
            if not tf.isVignetted and not tb.isVignetted and not spider.contains(tf.x0, tf.y0):
                fpx.append(tb.x0)
                fpy.append(tb.y0)
                pmx.append(tf.x0)
                pmy.append(tf.y0)
    return fpx, fpy, pmx, pmy

In [ ]:
def pupil(x, y):
    scale = 0.168 # arcsec / pixel
    scale /= 15e-6 # arcsec / meter
    scale /= 206265 # radians / meter
    theta_x = -x * scale
    theta_y = -y * scale
    print("Field angle", theta_x*180/np.pi, theta_y*180/np.pi)

    rays = batoid.parallelRays(
        z=1, outer=4.3, inner=0.0, theta_x=theta_x, theta_y=theta_y, wavelength=750e-9,
        medium=batoid.ConstMedium(1.0), nradii=100, naz=512
    )
    fullRays = HSC.traceFull(rays)
    pmRV = fullRays[0]['outrays']
    fpRV = fullRays[-1]['outrays']
    fpRVarray = np.array(fpRV)
    w = np.logical_not(fpRVarray['isVignetted'])
    print(np.mean(fpRVarray['p0']['x'][w]), np.mean(fpRVarray['p0']['y'][w]))
    w = spider.contains(pmRV.x, pmRV.y)
    return batoid.RayVector([r for r, w_, r2 in zip(pmRV, w, fpRV) if not w_ and not r2.isVignetted])

In [ ]:
def comparison_plot(x, y):
    fpx, fpy, pmx, pmy = pinhole(x, y)
    fpSpanX = np.max(fpx) - np.min(fpx)
    fpSpanY = np.max(fpy) - np.min(fpy)
    fpSpan = np.max([fpSpanX, fpSpanY])
    fpMidX = 0.5*(np.max(fpx)+np.min(fpx))
    fpMidY = 0.5*(np.max(fpy)+np.min(fpy))


    pupilRays = pupil(x, y)
    pupilSpanX = np.max(pupilRays.x) - np.min(pupilRays.x)
    pupilSpanY = np.max(pupilRays.y) - np.min(pupilRays.y)
    pupilSpan = np.max([pupilSpanX, pupilSpanY])
    pupilMidX = 0.5*(np.max(pupilRays.x)+np.min(pupilRays.x))
    pupilMidY = 0.5*(np.max(pupilRays.y)+np.min(pupilRays.y))

    # Overplot normalized PH/pupil
    nPHx = (fpx-fpMidX)/(0.5*fpSpan)
    nPHy = (fpy-fpMidY)/(0.5*fpSpan)
    nPux = (pupilRays.x-pupilMidX)/(0.5*pupilSpan)
    nPuy = (pupilRays.y-pupilMidY)/(0.5*pupilSpan)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
    ax.scatter(nPHx, nPHy, s=3, c='r', alpha=0.5, label='pinhole')
    ax.scatter(nPux, nPuy, s=3, c='b', alpha=0.5, label='pupil')
    ax.legend()
    ax.set_aspect('equal')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_title("Focal plane position {}, {}".format(x, y))
    fig.show()

In [ ]:
comparison_plot(0.0, 0.0)

In [ ]:
comparison_plot(0.0, 0.1)

In [ ]:
comparison_plot(0.0, 0.2)

In [ ]:
comparison_plot(0.1, 0.1)